In [2]:
import pandas as pd

df = pd.read_csv(
    "../data/transactions.csv",
    encoding="latin1",
    on_bad_lines="skip"
)

df.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.39,17850,United Kingdom


In [8]:
print(df.shape)
print(df.columns)
print(df.info())
print(df.head(10))
print(df.describe())

(534531, 8)
Index(['InvoiceNo', ' StockCode   ', ' Description                           ',
       ' Quantity', ' InvoiceDate     ', ' UnitPrice', ' CustomerID',
       ' Country'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 534531 entries, 0 to 534530
Data columns (total 8 columns):
 #   Column                                   Non-Null Count   Dtype  
---  ------                                   --------------   -----  
 0   InvoiceNo                                534531 non-null  object 
 1    StockCode                               534531 non-null  object 
 2    Description                             534531 non-null  object 
 3    Quantity                                534531 non-null  int64  
 4    InvoiceDate                             534531 non-null  object 
 5    UnitPrice                               534531 non-null  float64
 6    CustomerID                              534531 non-null  object 
 7    Country                                 5

In [11]:
df_clean = df.copy()

In [12]:
df_clean.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.39,17850,United Kingdom


In [15]:
df_clean.columns

Index(['InvoiceNo', ' StockCode   ', ' Description                           ',
       ' Quantity', ' InvoiceDate     ', ' UnitPrice', ' CustomerID',
       ' Country'],
      dtype='object')

In [17]:
df_clean.columns = df_clean.columns.str.strip()

In [18]:
df_clean.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [19]:
df_clean = df_clean.dropna(subset=["CustomerID"])

In [20]:
df_clean = df_clean[
    (df_clean["Quantity"] > 0) &
    (df_clean["UnitPrice"] > 0)
]

In [22]:
df_clean["InvoiceDate"] = df_clean["InvoiceDate"].astype(str).str.strip()

In [23]:
df_clean["InvoiceDate"] = pd.to_datetime(
    df_clean["InvoiceDate"],
    dayfirst=True,
    errors="coerce"
)

In [24]:
df_clean = df_clean.dropna(subset=["InvoiceDate"])

In [25]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 522882 entries, 0 to 534530
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    522882 non-null  object        
 1   StockCode    522882 non-null  object        
 2   Description  522882 non-null  object        
 3   Quantity     522882 non-null  int64         
 4   InvoiceDate  522882 non-null  datetime64[ns]
 5   UnitPrice    522882 non-null  float64       
 6   CustomerID   522882 non-null  object        
 7   Country      522882 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 35.9+ MB


In [28]:
reference_date = df_clean["InvoiceDate"].max() + pd.Timedelta(days=1)

In [29]:
df_clean.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [30]:
df_clean["TotalAmount"] = df_clean["Quantity"] * df_clean["UnitPrice"]

In [31]:
df_clean[["Quantity", "UnitPrice", "TotalAmount"]].head()

,Quantity,UnitPrice,TotalAmount
0,6,2.55,15.30
1,6,3.39,20.34
2,8,2.75,22.00
3,6,3.39,20.34
4,6,3.39,20.34


In [32]:
reference_date = df_clean["InvoiceDate"].max() + pd.Timedelta(days=1)

rfm = (
    df_clean
    .groupby("CustomerID")
    .agg({
        "InvoiceDate": lambda x: (reference_date - x.max()).days,
        "InvoiceNo": "nunique",
        "TotalAmount": "sum"
    })
    .reset_index()
)

rfm.columns = ["CustomerID", "Recency", "Frequency", "Monetary"]

In [33]:
from sklearn.model_selection import train_test_split

X = rfm[["Recency", "Frequency"]]
y = rfm["Monetary"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [34]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)


LinearRegression()

In [35]:
y_pred = model.predict(X_test)

In [36]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

rmse, r2

(np.float64(8973.042467069943), -1.3524842054733113)

In [37]:
results = X_test.copy()
results["Actual_CLV"] = y_test
results["Predicted_CLV"] = y_pred

results.to_csv("../outputs/clv_predictions.csv", index=False)